In [1]:
## Reasoning 모델 사용해보기
#!pip install vllm 
!wget https://huggingface.co/unsloth/Qwen3-0.6B-GGUF/resolve/main/Qwen3-0.6B-Q2_K.gguf
!wget https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct-GGUF/resolve/main/qwen2.5-0.5b-instruct-q4_0.gguf

In [1]:
# pip install -U vllm transformers
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

# Create the LLM runner
llm = LLM(
    model='./Qwen3-0.6B-Q2_K.gguf',
    tokenizer="Qwen/Qwen3-0.6B",
    trust_remote_code=True,
    dtype="auto",
    max_model_len=6672,
    gpu_memory_utilization=0.6,
)


INFO 10-12 23:58:39 [__init__.py:216] Automatically detected platform cuda.
INFO 10-12 23:58:40 [utils.py:328] non-default args: {'tokenizer': 'Qwen/Qwen3-0.6B', 'trust_remote_code': True, 'max_model_len': 6672, 'gpu_memory_utilization': 0.6, 'disable_log_stats': True, 'model': './Qwen3-0.6B-Q2_K.gguf'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 10-12 23:59:09 [__init__.py:742] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


ERROR 10-12 23:59:09 [config.py:278] Error retrieving safetensors: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './Qwen3-0.6B-Q2_K.gguf'., retrying 1 of 2
ERROR 10-12 23:59:11 [config.py:276] Error retrieving safetensors: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './Qwen3-0.6B-Q2_K.gguf'.
INFO 10-12 23:59:12 [__init__.py:2764] Downcasting torch.float32 to torch.bfloat16.
INFO 10-12 23:59:12 [__init__.py:1815] Using max model len 6672
WARNING 10-12 23:59:12 [__init__.py:1217] gguf quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 10-12 23:59:29 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 10-12 23:59:31 [_ipex_ops.py:16] Import error msg: No module named 'intel_extension_for_pytorch'
(EngineCore_DP0 pid

[W1012 23:59:49.850064038 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=9764) INFO 10-12 23:59:49 [gpu_model_runner.py:2370] Loading model from scratch...
(EngineCore_DP0 pid=9764) INFO 10-13 00:00:01 [cuda.py:362] Using Flash Attention backend on V1 engine.
(EngineCore_DP0 pid=9764) INFO 10-13 00:00:09 [gpu_model_runner.py:2392] Model loading took 0.2909 GiB and 19.035000 seconds
(EngineCore_DP0 pid=9764) INFO 10-13 00:00:13 [backends.py:539] Using cache directory: /home/ssafy/.cache/vllm/torch_compi

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████████████████████| 67/67 [00:13<00:00,  4.82it/s]


(EngineCore_DP0 pid=9764) INFO 10-13 00:00:38 [gpu_model_runner.py:3118] Graph capturing finished in 15 secs, took 0.63 GiB
(EngineCore_DP0 pid=9764) INFO 10-13 00:00:38 [gpu_worker.py:391] Free memory on device (3.2/4.0 GiB) on startup. Desired GPU memory utilization is (0.6, 2.4 GiB). Actual usage is 0.29 GiB for weight, 1.39 GiB for peak activation, 0.0 GiB for non-torch memory, and 0.63 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=-73431348` to fit into requested memory, or `--kv-cache-memory=786820404` to fully utilize gpu memory. Current kv cache memory in use is 765429452 bytes.
(EngineCore_DP0 pid=9764) INFO 10-13 00:00:38 [core.py:218] init engine (profile, create kv cache, warmup model) took 29.82 seconds
INFO 10-13 00:00:40 [llm.py:295] Supported_tasks: ['generate']
INFO 10-13 00:00:40 [__init__.py:36] No IOProcessor plugins requested by the model


In [16]:
# Set decoding params
sampling_params = SamplingParams(
    temperature=0.6,
    top_p=0.95,
    max_tokens=4096,
    # repetition_penalty=1.01
)

# Compose chat messages
messages = [
    {"role": "system", "content": "You are a helpful Korean assistant."},
    {"role": "user", "content": "Wendi는 하루에 닭 한 마리당 세 컵의 혼합 먹이를 줍니다. 아침에 15컵, 오후에 25컵의 먹이를 줍니다. Wendi의 닭 무리가 총 20마리일 때, 마지막 식사로 몇 컵의 먹이를 줘야 하나요?"} # 정답은 402
]
# 정답은 20
output = llm.chat(messages, sampling_params=sampling_params)
text = output[0].outputs[0].text
print(text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|                      | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…

<think>
Okay, let's see. The problem says that Wendi is making a combination of 15 cups and 25 cups each day. The question is, when she's making her 15 cups, she has 20 cups left, and the question is how much is left for her? The problem is phrased as: "Wendi is making a combination of 15 cups and 25 cups each day. Wendi's total is 20 cups, and the question is how much is left for her?" 

Wait, the original question is phrased as "Wendi is making a combination of 15 cups and 25 cups each day. Wendi's total is 20 cups, and the question is how much is left for her?" 

But the user is asking for the answer, and the initial answer was "20 cups." But the user's question is phrased as: "Wendi is making a combination of 15 cups and 25 cups each day. Wendi's total is 20 cups, and the question is how much is left for her?" 

So the user is asking for the answer, but the initial answer was "20 cups." 

But the problem is phrased as: "Wendi is making a combination of 15 cups and 25 cups each day.

In [17]:

# Compose chat messages
messages = [
    {"role": "system", "content": "You are a helpful Korean assistant."},
    {"role": "user", "content": "잠실에서 신촌으로 가려면 어떻게 해야해?"}
]

output = llm.chat(messages, sampling_params=sampling_params)
text = output[0].outputs[0].text
print(text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|                      | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…

<think>
<think>
<think>
Okay, let's tackle this. The user is asking how to get to a village in the area "Jum" in Korea. First, I need to figure out what "Jum" refers to. It could be a specific area or a place that's popular in Korean culture. Since the user is asking about getting to a village in Korea, the answer should focus on the Korean village in Jum. 

The user might be looking for information on how to get to a village in Korea, perhaps in the context of a Korean village or a similar area. The answer should be structured, maybe in Korean, but since I'm writing in English, it should be clear and helpful.

Since the user is asking about the "Jum" area, the answer should mention the area's location, maybe its geography, and the challenges or ways to get there. Also, since it's a village, maybe including some specific details about the village in Jum would be helpful. 

It's important to mention that the answer is in Korean, but since the user is asking in English, the answer should

In [21]:
import litellm
import os
from litellm import completion

os.environ['OPENROUTER_API_KEY'] = ""


responses = completion(
    model="openrouter/qwen/qwen3-235b-a22b:free",
    messages=[{ "content": "1+1이 2임을 증명하시요.","role": "user"}]
)


In [23]:
# 오류가 나는 경우 재 실행 필요
think, response = responses.choices[0].message.content.split('</think>')

In [24]:
print(think)

<think>Okay, I need to prove that 1+1 equals 2. Let me think about how to approach this.

First, I remember that in mathematics, basic arithmetic operations like addition are defined based on axioms. The most common foundation for this is the Peano axioms, which define the natural numbers. Let me recall what those axioms are.

The Peano axioms start with 0 as a natural number, and then there's a successor function S(n) that gives the next number. So, 1 is S(0), 2 is S(1) which is S(S(0)), and so on.

Addition is defined recursively. The definition usually goes like this: for any natural number a, a + 0 = a, and a + S(b) = S(a + b).

So, applying this to 1 + 1. Since 1 is S(0), then 1 + 1 = 1 + S(0). Using the recursive definition, this should be S(1 + 0). And since a + 0 = a, 1 + 0 = 1, so S(1 + 0) = S(1) = 2.

Wait, let me check that again. Let's define 1 as S(0). Then, 1 + 1 = S(0) + S(0). According to the addition definition, a + S(b) = S(a + b). So here, a is S(0) and b is 0. So S(

In [25]:
print(response)



다음은 1 + 1 = 2임을 **페아노 공리계**(Peano axioms) 기반으로 수학적으로 증명한 과정입니다. 이는 자연수의 기초를 정의하는 표준적 방법입니다.

---

### **정의 및 준비**
1. **0을 자연수의 시작으로 정의**합니다.
2. **후임자 함수 $ S(n) $**를 도입합니다.  
   - $ S(n) $는 $ n $의 "다음 수"를 의미합니다.  
   - 예: $ S(0) = 1 $, $ S(1) = 2 $, $ S(2) = 3 $, ...
3. **덧셈 연산을 재귀적으로 정의**합니다.  
   - (a) 임의의 자연수 $ a $에 대해 $ a + 0 = a $.  
   - (b) 임의의 자연수 $ a, b $에 대해 $ a + S(b) = S(a + b) $.

---

### **증명 단계**
1. $ 1 $과 $ 2 $를 페아노 공리계로 표현:  
   - $ 1 \equiv S(0) $,  
   - $ 2 \equiv S(S(0)) $.

2. **$ 1 + 1 $ 계산**:  
   $$
   1 + 1 = S(0) + S(0)
   $$  
   덧셈의 재귀적 정의 (b)에 따라:  
   $$
   S(0) + S(0) = S\big(S(0) + 0\big)
   $$  

3. **$ S(0) + 0 $ 계산**:  
   덧셈의 정의 (a)에 따라:  
   $$
   S(0) + 0 = S(0) = 1
   $$  

4. **결과 대입**:  
   $$
   S\big(S(0) + 0\big) = S\big(S(0)\big) = S(1) = 2
   $$  

5. **최종 결론**:  
   $$
   1 + 1 = 2
   $$

---

### **핵심 논리**
- 이 증명은 **자연수의 공리적 정의**와 **덧셈의 재귀적 규칙**에 완전히 의존합니다.  
- 페아노 공리계는 "1 + 1 = 2"가 인간의 직관이 아닌 **수학적 엄밀성**에서 비롯됨을 보여줍니다.  
- 러셀과 화이트헤드의 《

In [26]:
import litellm
import os
from litellm import batch_completion_models

response = batch_completion_models(
    models=[
        "openrouter/deepseek/deepseek-r1-0528-qwen3-8b:free",
        "openrouter/openai/gpt-oss-20b:free",
        "openrouter/nvidia/nemotron-nano-9b-v2:free",
        "openrouter/qwen/qwen3-30b-a3b:free",
    ], 
    messages=[{"role": "user", "content": "prove 1+1"}]
)

In [27]:
import litellm
import os
from litellm import batch_completion

responses = batch_completion(
    model="openrouter/qwen/qwen3-30b-a3b:free",
    messages = [
        [
            {
                "role": "user",
                "content": "good morning? "
            }
        ],
        [
            {
                "role": "user",
                "content": "what's the time? "
            }
        ]
    ]
)